# Project 2: Supervised Learning
### Building a Student Intervention System

## 1. Classification vs Regression

Your goal is to identify students who might need early intervention - which type of supervised machine learning problem is this, classification or regression? Why?

**Answer:**

**This is a classification problem because it has a discrete number of output values. For this project, the output values are 1(student passed), and 0(student failed).**

## 2. Exploring the Data

Let's go ahead and read in the student dataset first.

_To execute a code cell, click inside it and press **Shift+Enter**._

In [2]:
# Import libraries
import numpy as np
import pandas as pd

In [3]:
# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"
# Note: The last column 'passed' is the target/label, all other are feature columns
df = pd.DataFrame(student_data)

Student data read successfully!


Now, can you find out the following facts about the dataset?
- Total number of students
- Number of students who passed
- Number of students who failed
- Graduation rate of the class (%)
- Number of features

_Use the code block below to compute these values. Instructions/steps are marked using **TODO**s._

In [4]:
# TODO: Compute desired values - replace each '?' with an appropriate expression/function call
n_students = np.size(df['passed'])
n_features = df.iloc[[0]].size - 1
n_passed   = df[df['passed'] == 'yes']['passed'].size
n_failed   = df[df['passed'] == 'no']['passed'].size
grad_rate  = n_passed/395.0*100

print "Total number of students: {}".format(n_students)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Number of features: {}".format(n_features)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of students who passed: 265
Number of students who failed: 130
Number of features: 30
Graduation rate of the class: 67.09%


## 3. Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Let's first separate our data into feature and target columns, and see if any features are non-numeric.<br/>
**Note**: For this dataset, the last column (`'passed'`) is the target or label we are trying to predict.

In [5]:
# Extract feature (X) and target (y) columns
feature_cols = list(student_data.columns[:-1])  # all columns but last are features
target_col   = student_data.columns[-1]         # last column is the target/label

print "Feature column(s):-\n{}".format(feature_cols)
print "Target column: {}".format(target_col)

X_all = student_data[feature_cols]  # feature values for all students
y_all = student_data[target_col]    # corresponding targets/labels

print "\nFeature values:-"
print X_all.head()  # print the first 5 rows

Feature column(s):-
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']
Target column: passed

Feature values:-
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...    

### Preprocess feature columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation.

In [6]:
from sklearn.preprocessing import MaxAbsScaler
import warnings

# Preprocess feature columns
def preprocess_features(X,y):
    outX = pd.DataFrame(index=X.index)  # output dataframe, initially empty

    # Check each column
    for col, col_data in X.iteritems():
        # If data type is non-numeric, try to replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])
        # Note: This should change the data type for yes/no columns to int

        # If still non-numeric, convert to one or more dummy variables
        if col_data.dtype == object:
            col_data = pd.get_dummies(col_data, prefix=col)  # e.g. 'school' => 'school_GP', 'school_MS'

        outX = outX.join(col_data)  # collect column(s) in output dataframe
    
    # Ignoring a deprecated function warning for now
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        outY = y.replace(['yes','no'],[1,0])
        outX = outX.apply(lambda x: MaxAbsScaler().fit_transform(x))
        print outX.head()
    
    return outX, outY

outX, outY = preprocess_features(X_all,y_all)

print "Processed feature columns ({}):-\n{}".format(len(outX.columns), list(outX.columns))

   school_GP  school_MS  sex_F  sex_M       age  address_R  address_U  \
0          1          0      1      0  0.818182          0          1   
1          1          0      1      0  0.772727          0          1   
2          1          0      1      0  0.681818          0          1   
3          1          0      1      0  0.681818          0          1   
4          1          0      1      0  0.727273          0          1   

   famsize_GT3  famsize_LE3  Pstatus_A    ...     higher  internet  romantic  \
0            1            0          1    ...          1         0         0   
1            1            0          0    ...          1         1         0   
2            0            1          0    ...          1         1         0   
3            1            0          0    ...          1         1         1   
4            1            0          0    ...          1         0         0   

   famrel  freetime  goout  Dalc  Walc  health  absences  
0     0.8       0.6  

### Split data into training and test sets

So far, we have converted all _categorical_ features into numeric values. In this next step, we split the data (both features and corresponding labels) into training and test sets.

In [7]:
from sklearn.cross_validation import train_test_split

def setupTrainTest(outX, outY, student_data):
    # First, decide how many training vs test samples you want
    num_all  = student_data.shape[0]  # same as len(student_data)
    num_train = 300                   # about 75% of the data
    num_test  = num_all - num_train

    # TODO: Then, select features (X) and corresponding labels (y) for the training and test sets
    # Note: Shuffle the data or randomly select samples to avoid any bias due to ordering in the dataset
    X_train, X_test, y_train, y_test = train_test_split(
                                            outX,
                                            outY,
                                            test_size=num_test,
                                            random_state=42)

    print "Training set: {} samples".format(X_train.shape[0])
    print "Test set: {} samples".format(X_test.shape[0])
    
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = setupTrainTest(outX, outY, student_data)
# Note: If you need a validation set, extract it from within training data

Training set: 300 samples
Test set: 95 samples


## 4. Training and Evaluating Models
Choose 3 supervised learning models that are available in scikit-learn, and appropriate for this problem. For each model:

- What is the theoretical O(n) time & space complexity in terms of input size?
- What are the general applications of this model? What are its strengths and weaknesses?
- Given what you know about the data so far, why did you choose this model to apply?
- Fit this model to the training data, try to predict labels (for both training and test sets), and measure the F<sub>1</sub> score. Repeat this process with different training set sizes (100, 200, 300), keeping test set constant.

Produce a table showing training time, prediction time, F<sub>1</sub> score on training set and F<sub>1</sub> score on test set, for each training set size.

Note: You need to produce 3 such tables - one for each model.

**Answer:**

**The complexity for Random Forest is O( nTrees X smplVarNode X n log(n) ) where nTrees is the number of decision trees you want to build, smplVarNode is number of sample variables tested for each node, and the n is the number of records being used. If you choose to restrict the depth of the trees, you can cut the complexity down to O( nTrees X smplVarNode X d X n ) where d is the max depth.**

**The complexity for SVM is O( nSmpl^2 X nFeatures ) where nSmpl is the number of samples being tested and nFeatures is the number of features(ie. variables) in your model.**

**The complexity for Perceptron is O( nEpochs X n X nFeatures X nNeurons ) where nEpochs is the number of epochs(ie. all training vectors used once to update the weights), n is the number of training examples, nFeatures is the number of features, and nNeurons is the number of neurons used in the model.**

**The general applications for the Random Forest model are both classification and regression tasks. It is one of the most accurate modeling algorithms and can work fairly well with most datasets. The pros of this model is that it can work efficiently with very large databases and generalizes quite well. It takes your data, then divides it and uses a different decision tree for each group. It then votes for what it thinks is the best classification or approximation so it is a little like how voting works. The cons of this model are that with particularly noisy data, it can overfit easier. Another weakness is that with few training examples, it has a hard time accurately approximating, especially if there are a lot of features in the dataset.**

**The general applications for the SVM model is classification problems. It is useful for both linear and non-linear classification problems. The pros of this model is that it can work with small datasets quite well, even when there are a lot of features in that dataset. So while Random Forest does worse on small datasets, SVM does better and vice versa. The cons with SVM is that firstly, it is only a classifier, and doesn't do regression, and secondly that if the problem is non-linear, the computation cost rises significantly. This problem can become evident when the dataset is huge so it's generally better to use a different model for huge datasets.**

**The general applications for the Perceptron Network model is classification and regression. It's a relatively simple model for solving linear problems in a timely fashion. The Perceptron was created to function a little like a neuron in your brain does. It's only one layer so it can't get into extremely complicated like its upgraded multi-layered perceptron can, but for our example there's no need. The Perceptron acts like a human brain in that each node is equivalent to a neuron in the brain, and when necessary you can add layers on to make an MLP. This allows us to solve unbelievably complex problems and is at the forefront of machine learning(as showcased by AlphaGO, Google's Go playing algorithm that used a deep neural network(multiple hidden layers), in combination with as a Monte-Carlo decision tree algorithm, and beat the world champian several weeks ago. Go has up to 10^176 possible combination states so it goes to show what this type of algorithm is capable of). The pros of the Perceptron as well as the MLP algorithm is solving both simple problems with the Perceptron and incredibly complex problems with the MLP. The cons is the training time. The training time can take a VERY long time because of the complexity of the problems it's usually solving. However what you pay up front in training time is made up for with quick testing times. **

**The reason I chose these three models was because given that there was a great many features to deal with and relatively few training examples, I needed models that could work with a high feature set. Since we were dealing with classification, it obviously made any regression algorithms useless, so I was left to choose from classification algorithms. Also due to the heavy feature set, I wanted classifiers that could deal in the non-linear solutions. But I also wanted to have a variety to choose from. For example, I knew SVM worked well with very few training examples, but I wasn't sure how many "few" was and if the 300 examples was enough to classify as more than a few. That's why I chose Random Forest as it works well with greater number of training examples and is generally a very accurate model to work with. The last algorithm I chose was the Perceptron. The reason for this was to get the feel of using neuron network in preparation for using a multi-layered perceptron network, or deep learning networks, as it's very important in the real world. I wasn't exactly sure it would be the best fit considering the data but I believed it was certainly worth trying.**

In [22]:
# Train a model
import time

# Trains 3 seperate classifier models, SVC, Perceptron, and Random Forest
# on the same data to find out which is the optimal strategy to use
# Also included is the timing for each training 
def train_classifier(clfs, X_train, y_train):
    
    print "Training {}...".format(clfs.__class__.__name__),"\n"
    
    svc_clf = clfs[0]
    perc_clf = clfs[1]
    rf_clf = clfs[2]
    
    start = time.time()
    svc_clf.fit(X_train, y_train)
    end = time.time()
    
    start2 = time.time()
    np_X = X_train.as_matrix() 
    np_y = y_train.as_matrix()
    perc_clf.fit(np_X, np_y)
    end2 = time.time()
   
    start3 = time.time()
    rf_clf.fit(X_train, y_train)
    end3 = time.time()
    
    svc_time  = end-start
    perc_time = end2-start2
    rf_time   = end3-start3
    
    time_data = [svc_time, perc_time, rf_time]
    trained_models = [svc_clf, perc_clf, rf_clf]
    
    print "\nTraining time SVM (secs): {:.3f}".format(svc_time)
    print "Training time Perceptron (secs): {:.3f}".format(perc_time)
    print "Training time RandomForest (secs): {:.3f}".format(rf_time)
    
    return time_data, trained_models

# TODO: Choose a model, import it and instantiate an object
from sklearn.linear_model import Perceptron
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

svc_clf  = SVC(kernel='rbf')
perc_clf = Perceptron(n_iter=50)
rf_clf   = RandomForestClassifier(n_estimators=45)

clfs = [svc_clf, perc_clf, rf_clf]

# Fit model to training data
time_data, trained_models = train_classifier(clfs, X_train, y_train)  # note: using entire training set here
#print clf  # you can inspect the learned model by printing it

Training list... 


Training time SVM (secs): 0.016
Training time Perceptron (secs): 0.000
Training time RandomForest (secs): 0.094


In [23]:
# Predict on training set and compute F1 score
from sklearn.metrics import f1_score, make_scorer
from sklearn.grid_search import GridSearchCV

def performance_metric(target, svc_pred, perc_pred, rf_pred):
    
    svc_f1_error  = f1_score(target.values, svc_pred)
    perc_f1_error = f1_score(target.values, perc_pred)
    rf_f1_error   = f1_score(target.values, rf_pred)
    
    f1_data = [svc_f1_error, perc_f1_error, rf_f1_error]
    
    return f1_data

# Make predictions based on the 3 seperate classifiers to discover
# which returns a more accurate result. Also time each to discover 
# what takes the longest/shortest.
def predict_labels(clfs, features, target):
    
    print "\n","Predicting labels using {}...".format(clfs.__class__.__name__)
    
    svc_clf = clfs[0]
    perc_clf = clfs[1]
    rf_clf = clfs[2]
    
    start = time.time()
    svc_pred = svc_clf.predict(features)
    end = time.time()
    
    start2 = time.time()
    np_X = features.as_matrix()
    perc_pred = perc_clf.predict(np_X)
    end2 = time.time()
    
    start3 = time.time()
    rf_pred = rf_clf.predict(features)
    end3 = time.time()
    
    svc_pred_time  = end-start
    perc_pred_time = end2-start2
    rf_pred_time   = end3-start3
    
    prediction_time_data = [svc_pred_time, perc_pred_time, rf_pred_time]
    
    print "Prediction time SVM (secs): {:.3f}".format(svc_pred_time)
    print "Prediction time Perceptron (secs): {:.3f}".format(perc_pred_time)
    print "Prediction time RandomForest (secs): {:.3f}".format(rf_pred_time),"\n"
    
    f1_data = performance_metric(target, svc_pred, perc_pred, rf_pred)
    
    return prediction_time_data, f1_data

prediction_time_data, f1_data = predict_labels(trained_models, X_train, y_train)

print "F1 score for training set(SVM): {}".format(f1_data[0])
print "F1 score for training set(Perceptron): {}".format(f1_data[1])
print "F1 score for training set(RandomForest): {}".format(f1_data[2])


Predicting labels using list...
Prediction time SVM (secs): 0.016
Prediction time Perceptron (secs): 0.000
Prediction time RandomForest (secs): 0.000 

F1 score for training set(SVM): 0.811881188119
F1 score for training set(Perceptron): 0.829268292683
F1 score for training set(RandomForest): 1.0


In [24]:
# Predict on test data
prediction_time_data, f1_data = predict_labels(trained_models, X_test, y_test)

print "F1 score for test set(SVM): {}".format(f1_data[0])
print "F1 score for test set(Perceptron): {}".format(f1_data[1])
print "F1 score for test set(RandomForest): {}".format(f1_data[2])


Predicting labels using list...
Prediction time SVM (secs): 0.000
Prediction time Perceptron (secs): 0.000
Prediction time RandomForest (secs): 0.000 

F1 score for test set(SVM): 0.774193548387
F1 score for test set(Perceptron): 0.746666666667
F1 score for test set(RandomForest): 0.791666666667


In [25]:
# Train and predict using different training set sizes
def train_predict(clfs, X_train, y_train, X_test, y_test):
    
    print "------------------------------------------"
    print "Training set size: {}".format(len(X_train))
    
    training_time_data, trained_models = train_classifier(clfs, X_train, y_train)
    
    training_pred_time_data, training_f1_error_data = predict_labels(trained_models, X_train, y_train)
    
    print "F1 score for training set(SVM): {}".format(training_f1_error_data[0])
    print "F1 score for training set(Perceptron): {}".format(training_f1_error_data[1])
    print "F1 score for training set(RandomForest): {}".format(training_f1_error_data[2])
    
    testing_pred_time_data, testing_f1_error_data = predict_labels(trained_models, X_test, y_test)
    
    print "F1 score for test set(SVM): {}".format(testing_f1_error_data[0])
    print "F1 score for test set(Perceptron): {}".format(testing_f1_error_data[1])
    print "F1 score for test set(RandomForest): {}".format(testing_f1_error_data[2])
    
    table_data = [training_time_data[0],testing_pred_time_data[0],training_f1_error_data[0], testing_f1_error_data[0],
                  training_time_data[1],testing_pred_time_data[1],training_f1_error_data[1],testing_f1_error_data[1],
                  training_time_data[2],testing_pred_time_data[2],training_pred_error_data[2],testing_f1_error_data[2]]
    return table_data
# TODO: Run the helper function above for desired subsets of training data
# Note: Keep the test set constant
td300 = train_predict(trained_models,X_train,y_train,X_test,y_test)
td200 = train_predict(trained_models,X_train[:200],y_train[:200],X_test,y_test)
td100 = train_predict(trained_models,X_train[:100],y_train[:100],X_test,y_test)

------------------------------------------
Training set size: 300
Training list... 


Training time SVM (secs): 0.016
Training time Perceptron (secs): 0.000
Training time RandomForest (secs): 0.112

Predicting labels using list...
Prediction time SVM (secs): 0.004
Prediction time Perceptron (secs): 0.000
Prediction time RandomForest (secs): 0.004 

F1 score for training set(SVM): 0.811881188119
F1 score for training set(Perceptron): 0.829268292683
F1 score for training set(RandomForest): 1.0

Predicting labels using list...
Prediction time SVM (secs): 0.004
Prediction time Perceptron (secs): 0.000
Prediction time RandomForest (secs): 0.004 

F1 score for test set(SVM): 0.774193548387
F1 score for test set(Perceptron): 0.746666666667
F1 score for test set(RandomForest): 0.791666666667
------------------------------------------
Training set size: 200
Training list... 


Training time SVM (secs): 0.008
Training time Perceptron (secs): 0.000
Training time RandomForest (secs): 0.168

Predic

In [27]:
# Make this above data easier to read and understand by converting
# to a pandas dataframe.
svm = ["Training time(secs),SVM:","Prediction time (secs),SVM:",
       "F1 score for training set,SVM","F1 score for test set,SVM"]   
p   = ["Training time(secs),P:","Prediction time (secs),P:",
       "F1 score for training set,P","F1 score for test set,P"]    
rf  = ["Training time(secs),RF:","Prediction time (secs),RF:",
       "F1 score for training set,RF","F1 score for test set,RF",]

svm_data  = {'100 Samples': td100[:4], '200 Samples': td200[:4], '300 Samples':td300[:4]}
perc_data = {'100 Samples': td100[4:8], '200 Samples': td200[4:8], '300 Samples':td300[4:8]}
rf_data   = {'100 Samples': td100[8:12], '200 Samples': td200[8:12], '300 Samples':td300[8:12]}

svm_table = pd.DataFrame(data=svm_data, index=svm)
p_table = pd.DataFrame(data=perc_data, index=p)
rf_table = pd.DataFrame(data=rf_data, index=rf)

print svm_table,"\n"
print p_table,"\n"
print rf_table

                               100 Samples  200 Samples  300 Samples
Training time(secs),SVM:          0.004000     0.008000     0.016000
Prediction time (secs),SVM:       0.004000     0.000000     0.004000
F1 score for training set,SVM     0.780488     0.816568     0.811881
F1 score for test set,SVM         0.774194     0.774194     0.774194 

                             100 Samples  200 Samples  300 Samples
Training time(secs),P:          0.000000     0.000000     0.000000
Prediction time (secs),P:       0.000000     0.000000     0.000000
F1 score for training set,P     0.810127     0.533333     0.829268
F1 score for test set,P         0.783784     0.687500     0.746667 

                              100 Samples  200 Samples  300 Samples
Training time(secs),RF:          0.156000     0.168000     0.112000
Prediction time (secs),RF:       0.000000     0.004000     0.004000
F1 score for training set,RF     1.000000     1.000000     1.000000
F1 score for test set,RF         0.760563   

## 5. Choosing the Best Model

- Based on the experiments you performed earlier, in 1-2 paragraphs explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?
- In 1-2 paragraphs explain to the board of supervisors in layman's terms how the final model chosen is supposed to work (for example if you chose a Decision Tree or Support Vector Machine, how does it make a prediction).
- Fine-tune the model. Use Gridsearch with at least one important parameter tuned and with at least 3 settings. Use the entire training set for this.
- What is the model's final F<sub>1</sub> score?

**Answer:**

**Based on the experiments, the model I chose to best fit your needs would be the Random Forest model. Random Forest is your best option for a few reasons. First, it's relatively high average performance does better than SVM and the Perceptron algorithms on average. Although it ran slower than the SVM network, because our dataset used so little data, the slightly higher computation cost was worth it for the higher performance. It can be trained in a timely fashion and it doesn't need a ton of available data to receive good output. The seconds reason this option is best is because it is one of the best and most reliable algorithms available. It's a widely used algorithm that can predict with one of the best accuracies possible as long as it has enough data to train on. In your case, it does, although going much lower than 100 samples could be an issue. For these reasons, Random Forest is your best option for a high accuracy, high reliability solution for our problem that you're looking for.**

**You may be asking yourself, how does this model actually work? Now consider an example, suppose you ask me to go to a restaurant. I'm quite indecisive, so I ask if I'll like the restaurant. In order to decide if I'll like it, you'll have me give you a set of example restaurants and tell you whether I like it or not. Then when I ask you whether I'll like the new restaurant, you play 20 questions with me. You ask me questions like is it an Italian restaurant, is it a buffet, is it usually crowded, etc.. You start off with broad questions first to maximize the information gain, much like in the real game of 20 questions, and eventually get into more and more detailed questions. Eventually you give me a yes or no answer to whether I'll like it.**

**However, you're not perfect and you may have not generalized well enough to get perfect answers to new restaurants, so you have your friends play the same 20 question game with me but they choose different questions to ask me to get there prediction and you also give them a different set of examples that I liked or did not like. Now you and each of your friends will now vote whether I would or would not like this new restaurant and the prediction that is highest wins out. Now consider this example but with our data. The 300 student data examples represents the list of restaurants I told you I liked or disliked. You split up that dataset and gave each set of data to one of your friends. For each dataset, your friends randomly choose a set of variables, like sex, age, family size, and use those variables to formulate there questions for me. They each then vote yes/no or in our case pass/fail, and the one with the highest amount of votes wins. This is what Random Forest does.**

**I ended up with a 80.3% F-score.**

In [37]:
# TODO: Fine-tune your model and report the best F1 score
rf_clf = RandomForestClassifier()

f1_error = make_scorer(f1_score)

params = {'n_estimators':[30,35,40,45,50,55,60],
          'max_features':["auto",7,8],
          'max_depth':[None]}

rf_clf = GridSearchCV(estimator=rf_clf,
                   param_grid=params,
                   scoring=f1_error)

print "Training {}...".format(rf_clf.__class__.__name__),"\n"
start = time.time()
rf_clf.fit(X_train, y_train)
print rf_clf.best_estimator_
end = time.time()
print "Done!\nTraining time (secs): {:.3f}\n".format(end - start)

rf_pred = rf_clf.predict(X_train)
f1_error = f1_score(y_train.values, rf_pred)
print "Training F1_Score Error: {:.3f}".format(f1_error)

rf_test_pred = rf_clf.predict(X_test)
f1_test_error = f1_score(y_test.values, rf_test_pred)
print "Testing F1_Score Error: {:.3f}".format(f1_test_error)

Training GridSearchCV... 

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=60, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
Done!
Training time (secs): 5.336

Training F1_Score Error: 1.000
Testing F1_Score Error: 0.803
